In [1]:
import pandas as pd
import numpy as np
import sys
def drop_duplicates(df,column='z'):
    tmp=df.copy()
    tmp['zstr']=tmp[column].astype(str)#.drop_duplicates(subset='z')
    tmp=tmp.drop_duplicates(subset='zstr')
    return tmp.drop('zstr',axis='columns')

In [2]:
REBUILD=False

In [3]:
if not REBUILD:
    sys.exit('Ignore this cell')
dfn=pd.read_json('s9.json')

dfn['n']=dfn['z'].apply(lambda l: 8 if l.count(0)==1 else 9)

if dfn[dfn['z'].apply(lambda l: l.count(0))>1].empty:
    print('Ready n')
    kk=dfn['z'].apply(lambda l: l.remove(0) if l.count(0)==1 else l )

assert dfn.shape[0]==dfn['z'].astype(str).unique().shape[0]

for n in [8,7,6]:
    pdfn={}
    pdfn[n]=pd.read_json('s{}.json'.format(n))

    assert pdfn[n].shape[0]==pdfn[n]['z'].astype(str).unique().shape[0]

    pdfn[n]['n']=pdfn[n]['z'].apply(lambda l: n-1 if l.count(0)==1 else n)

    if pdfn[n][pdfn[n]['z'].apply(lambda l: l.count(0))>1].empty:
        print('Ready n')
        kk=pdfn[n]['z'].apply(lambda l: l.remove(0) if l.count(0)==1 else l )
    print(n,'→',pdfn[n][pdfn[n]['n']==n].shape)
    dfn=dfn.append(pdfn[n]).reset_index(drop=True)
    
    #Reordering:
    dfn['ls']=dfn['l'].apply(lambda l: np.abs(l).max())
    dfn['ks']=dfn['k'].apply(lambda l: np.abs(l).max())
    dfn=dfn.sort_values(['ls','ks']).reset_index(drop=True)
    dfn=dfn.drop(['ls','ks'],axis='columns')
    #dfn=dfn.sort_values(['gcd']).reset_index(drop=True)
    dfn=drop_duplicates(dfn,column='z')
    print(n,'→',dfn[dfn['n']==n].shape)

SystemExit: Ignore this cell

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
if not REBUILD:
    sys.exit('Ignore this cell')
dfn[dfn['z'].apply(lambda l: l.count(0)>0)]

SystemExit: Ignore this cell

In [5]:
if not REBUILD:
    sys.exit('Ignore this cell')
rng={}
for n in [6,7,8,9]:
    rng[n]=[(dfn[dfn['n']==n].l.apply(lambda l: max(l)).max(),dfn[dfn['n']==n].l.apply(lambda l: min(l)).min()),
 (dfn[dfn['n']==n].k.apply(lambda l: max(l)).max(),dfn[dfn['n']==n].k.apply(lambda l: min(l)).min())]

SystemExit: Ignore this cell

`
{6: [(21, -21), (21, -21)],
  7: [(16, -16), (16, -16)],
  8: [(15, -15), (15, -15)],
  9: [(12, -12), (12, -12)]}
`

In [6]:
if not REBUILD:
    sys.exit('Ignore this cell')
rng

SystemExit: Ignore this cell

In [7]:
if not REBUILD:
    sys.exit('Ignore this cell')
dfn=dfn.rename(columns={'z':'solution'})
dfn.to_json('solutions.json',orient='records')

SystemExit: Ignore this cell

In [8]:
import os
if os.path.exists('/content'):
    !git clone https://github.com/restrepo/anomaly.git 2> /dev/null > /dev/null
    ! pip install anomalies 2> /dev/null > /dev/null
    os.chdir('anomaly')

Inehirate from `solutions` class to explore for solutions with repeated charges

## Find solutions with repeated charges

In [9]:
ds=pd.read_json('solutions.json')

In [10]:
ds.shape

(31211, 5)

In [11]:
ds['strz']=ds['solution'].astype(str)
ds.drop_duplicates('strz').shape

(31211, 6)

In [12]:
ds=ds.drop('strz',axis='columns')

In [13]:
ds[ds['solution'].apply(lambda l: len(l)-len(set(l)))>0].shape[0]

16805

## Check phenomenological conditions
$$ 2l+\nu-m=0$$

In [14]:
#See anomalysolutions.ipynb for details
from anomalytools import *

## Filter solutions with at most two massless chiral fermions

In [21]:
cl=pd.read_json('cl.json').reset_index(drop=True)
cl['sltn']=cl['sltn'].apply(jsonKeys2int)
cl=cl.sort_values('n').reset_index(drop=True)
cl6=pd.read_json('cl6.json').reset_index(drop=True)
cl6['sltn']=cl6['sltn'].apply(jsonKeys2int)
cl6=cl6.sort_values('n').reset_index(drop=True)

DD analysis

In [15]:
cl=ds.copy()
cl['sltn']=cl['solution'].astype(str)
cl['sltn']=cl['sltn'].apply(jsonKeys2int)
cl=cl.sort_values('n').reset_index(drop=True).drop('sltn',axis='columns')

In [18]:
def tree_level(ll):
    '''
    TODO: Only a few possibilities implemented
    '''
    ms=[]
    rp=[]
    for x in ll:
        if ll.count(x)==1:
            ms.append(x)
        elif ll.count(x)>1:
            rp.append(x)
    rp=sorted(np.unique(np.abs(rp)))
    TL=False

    if len(rp)==2:
        l=rp[0]
        if ll.count(l)<2:
            l=-l
        ν=rp[1]
        if ll.count(ν)<2:
            ν=-ν
        for m in ms:
            if m-2*l==ν:
                #print(ll,m,l,ν)
                TL=True
                break
    #elif len(rp)==3...
    return TL
TL=cl[cl['solution'].apply(tree_level)].reset_index(drop=True)

In [17]:
TL

,l,k,solution,gcd,n
0,"[1, -2]","[-4, 1]","[1, -4, -4, 9, 9, -11]",3,6
1,"[3, 1]","[-1, -5, 7]","[2, 2, -4, 7, -8, -8, 9]",1,7
2,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,7
3,"[-1, -2, 2]","[-10, -1, -8]","[1, 10, 10, 10, -19, -19, -19, 26]",17,8
4,"[3, 2, 1]","[-8, -13, -2]","[3, 7, -8, -8, -12, 23, 23, -28]",2,8
...,...,...,...,...,...
194,"[4, 1, 4]","[-1, -5, -4, -3]","[4, 4, -7, -8, 9, 11, -16, -16, 19]",1,9
195,"[4, -1, -2]","[-1, -4, -1, -4]","[3, 7, 7, -8, -9, 20, -22, -22, 24]",2,9
196,"[2, -4, 2]","[-1, -2, -1, 7]","[7, 7, -15, -16, 17, 17, -18, -30, 31]",8,9
197,"[2, -4, -1]","[-4, -8, -6, -1]","[1, -2, 4, 8, 8, -10, -12, -12, 15]",2,9
